In [1]:
import pandas as pd
import numpy as np
from itertools import product
import seaborn as sns
import os
import matplotlib.pyplot as plt
import scipy.sparse 
import sklearn

from utils import *

%matplotlib inline
data_path = 'data/'
seed=1204



In [2]:
for p in [np, pd, scipy, sklearn]:
    print (p.__name__, p.__version__)

numpy 1.13.3
pandas 0.20.3
scipy 0.19.1
sklearn 0.19.1


# Generating new_sales.csv

In [3]:
sales    = pd.read_csv(os.path.join(data_path, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(data_path, 'items.csv'),encoding ='ISO-8859-1')
item_categories = pd.read_csv(os.path.join(data_path, 'item_categories.csv'),encoding ='ISO-8859-1')
shops           = pd.read_csv(os.path.join(data_path, 'shops.csv'),encoding ='ISO-8859-1')
test            = pd.read_csv(os.path.join(data_path, 'test.csv.gz'))

In [4]:
sales = sales[sales.item_cnt_day<=1000] # there is only 1 item
sales = sales[sales.item_price<100000]

In [6]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates to get rid of duplicates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})



C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [7]:
# Fix column names
print(gb.columns.values)
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 

[('shop_id', '') ('item_id', '') ('date_block_num', '')
 ('item_cnt_day', 'target')]


In [8]:
gb.head()

,shop_id,item_id,date_block_num,target
0,0,30,1,31.0
1,0,31,1,11.0
2,0,32,0,6.0
3,0,32,1,10.0
4,0,33,0,3.0


In [9]:
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)


In [10]:
# merge with category id
all_data =pd.merge(all_data,items,on=['item_id'],how='left')

In [14]:

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'shop_block_target_sum':'sum','shop_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)


C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [16]:
# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item_block_target_sum':'sum','item_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [20]:
# Same as above but with item category-month aggregates
sales =pd.merge(sales,items,on=['item_id'],how='left')
gb = sales.groupby(['item_category_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item_cat_block_target_sum':'sum','item_cat_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_category_id', 'date_block_num']).fillna(0)

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [ ]:

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
# gc.collect();

In [26]:
all_data.shape

(10913804, 12)

In [4]:
all_data.to_csv(data_path+'new_sales.csv',index=False)


# ------------------------------------------

# Generate lag feature new_sales_lag_after12.pickle

In [ ]:
all_data = pd.read_csv(data_path+'new_sales.csv')

In [5]:
index_cols = ['shop_id', 'item_id', 'date_block_num','item_category_id']
cols_to_rename = list(all_data.columns.difference(index_cols))
for i in ['item_name']:
    cols_to_rename.remove(i)
print(cols_to_rename)
cols_gb_item = [i for i in cols_to_rename if 'item_block' in i]
cols_gb_shop = [i for i in cols_to_rename if 'shop_block' in i]
cols_gb_cat = [i for i in cols_to_rename if 'item_cat' in i]
cols_gb_all = ['target']
cols_gb_key=[['item_id'],['shop_id'],['item_category_id'],['shop_id','item_id']]
cols_gb_value = [cols_gb_item,cols_gb_shop,cols_gb_cat,cols_gb_all]
print(cols_gb_value)

['item_block_target_mean', 'item_block_target_sum', 'item_cat_block_target_mean', 'item_cat_block_target_sum', 'shop_block_target_mean', 'shop_block_target_sum', 'target']


In [6]:
shift_range = [1,2,3,5,12]
for month_shift in shift_range:
    for k,v in zip(cols_gb_key,cols_gb_value): 
        index_col = ['date_block_num'] + k
        train_shift = all_data[index_col + v].copy().drop_duplicates()

        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift

        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in v else x
        train_shift = train_shift.rename(columns=foo)
        all_data = pd.merge(all_data, train_shift, on=index_col, how='left').fillna(0)

In [10]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,...,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
0,59,22154,0,1.0,SCENE 2012 (BD),37,2017.0,1.092041,18.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,59,2552,0,0.0,DEEP PURPLE The House Of Blue Light LP,58,2017.0,1.092041,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,59,2554,0,0.0,DEEP PURPLE Who Do You Think We Are LP,58,2017.0,1.092041,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,59,2555,0,0.0,DEEP PURPLE 30 Very Best Of 2CD (Firm).,56,2017.0,1.092041,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,59,2564,0,0.0,DEEP PURPLE Perihelion: Live In Concert DVD (C...,59,2017.0,1.092041,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
all_data.shape

(10913804, 47)

In [11]:
# all_data.to_csv(data_path+'new_sales_lag.csv',index=False)

In [18]:
all_data = all_data[all_data['date_block_num'] >= 12] 

In [23]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,...,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
4488710,54,10297,12,4.0,THE POOR I AM 1-2 (BD),37,8198.0,1.295717,23.0,1.000000,...,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0
4488711,54,10296,12,3.0,THE POOR I AM 1-2 (3D BD),38,8198.0,1.295717,17.0,1.000000,...,1.023669,1384.0,0.0,0.0,0.0,1.247143,9386.0,1.019387,1630.0,0.0
4488712,54,10298,12,14.0,DESPICABLE ME 2,40,8198.0,1.295717,182.0,1.181818,...,1.102380,24130.0,0.0,0.0,0.0,1.247143,9386.0,1.145903,33489.0,0.0
4488713,54,10300,12,3.0,THE POOR I AM 2 (BD),37,8198.0,1.295717,26.0,0.962963,...,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0
4488714,54,10284,12,1.0,Vysotsky Vladimir Thank you that the live (mp3...,57,8198.0,1.295717,3.0,1.000000,...,1.005348,1316.0,0.0,0.0,0.0,1.247143,9386.0,1.003914,1539.0,0.0


In [20]:
# all_data.to_csv(data_path+'new_sales_lag_after12.csv',index=False)

# Add boolean holiday features

In [25]:
all_data['December'] = all_data.date_block_num.apply(lambda x: 1 if x ==23 else 0)

/home/quantran/anaconda3/envs/xgb/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
all_data['Newyear_Xmas'] = all_data.date_block_num.apply(lambda x: 1 if x in [12,24] else 0)
all_data['Valentine_MenDay'] = all_data.date_block_num.apply(lambda x: 1 if x in [13,25] else 0)
all_data['WomenDay'] = all_data.date_block_num.apply(lambda x: 1 if x in [14,26] else 0)
all_data['Easter_Labor'] = all_data.date_block_num.apply(lambda x: 1 if x in [15,27] else 0)

/home/quantran/anaconda3/envs/xgb/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/quantran/anaconda3/envs/xgb/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/quantran/anaconda3/envs/xgb/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [28]:
all_data.to_pickle(data_path+'new_sales_lag_after12.pickle')

In [30]:
all_data = pd.read_pickle(data_path+'new_sales_lag_after12.pickle')

In [31]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,...,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,December,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor
4488710,54,10297,12,4.0,THE POOR I AM 1-2 (BD),37,8198.0,1.295717,23.0,1.000000,...,1.247143,9386.0,1.053414,6094.0,0.0,0,1,0,0,0
4488711,54,10296,12,3.0,THE POOR I AM 1-2 (3D BD),38,8198.0,1.295717,17.0,1.000000,...,1.247143,9386.0,1.019387,1630.0,0.0,0,1,0,0,0
4488712,54,10298,12,14.0,DESPICABLE ME 2,40,8198.0,1.295717,182.0,1.181818,...,1.247143,9386.0,1.145903,33489.0,0.0,0,1,0,0,0
4488713,54,10300,12,3.0,THE POOR I AM 2 (BD),37,8198.0,1.295717,26.0,0.962963,...,1.247143,9386.0,1.053414,6094.0,0.0,0,1,0,0,0
4488714,54,10284,12,1.0,Vysotsky Vladimir Thank you that the live (mp3...,57,8198.0,1.295717,3.0,1.000000,...,1.247143,9386.0,1.003914,1539.0,0.0,0,1,0,0,0


In [32]:
all_data.shape

(6425094, 52)

# Add months feature (For Neural network only. This is not useful for Gradient Boosting)

In [ ]:
# # convert dbn to month
# months = [i for i in range(12)]
# month_list = [[j for j in range(i,34,12) if j >=12 ] for i in months]
# print(month_list)


# def get_month_from_dbn(date_block_num):
#     result=[]
#     for i,dbn in enumerate(date_block_num):
#         for j,m in enumerate(month_list):
#             if dbn in m:
#                 result.append(j+1)
#                 break
#     result = np.array(result)
#     return result

# all_data = pd.read_pickle(data_path+'new_sales_lag_after12.pickle')
# all_data['month'] = get_month_from_dbn(all_data.date_block_num)


# all_data.to_pickle(data_path+'new_sales_lag_after12_month.pickle')